# Ban User 목록 불러오기

In [2]:
import requests

# Banned Player List 가져오기
url = "https://api.gametools.network/"
response = requests.get(url+"bfban/banned_players/")

banned_user_ids = []
if response.status_code == 200:
    # 성공적으로 데이터를 받아옴
    data = response.json() # JSON 응답을 파이썬 딕셔너리로 변환
    banned_user_ids = data["personaids"]
else:
    # 오류 처리
    print(f'Error: {response.status_code}')

# Ban User Stat 불러오기

In [3]:
import pandas as pd
from tqdm import tqdm
import time
import requests
import os
from datetime import datetime

data_df = pd.DataFrame()
url = "https://bfvhackers.com/api/v1/is-hacker?player-id="

conn_cnt = 0
start_time = time.time()

# 백업 디렉토리 생성
backup_dir = "data_backups"
if not os.path.exists(backup_dir):
    os.makedirs(backup_dir)

for idx, id in enumerate(tqdm(banned_user_ids)):
    while True:
        try:
            # URL에 접근하여 데이터 불러옴
            response = requests.get(url + banned_user_ids[conn_cnt] + "&stats=false")
            conn_cnt = idx

            if response.status_code == 200:
                data = response.json()
                data = pd.json_normalize(data)
                data_df = pd.concat([data_df, data])
                
                # 데이터를 수집한 후 백업
                backup_file = os.path.join(backup_dir, f"data_backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv")
                data.to_csv(backup_file, index=False)
                
                # 데이터를 수집하고 난 뒤에는, sleep을 통해 종료를 방지
                conn_cnt += 1
                if conn_cnt % 10000 == 0:
                    time.sleep(600)
                else:
                    time.sleep(3)
                break
            
            elif response.status_code == 404:
                time.sleep(2)
                break  # 404 오류는 재시도하지 않고 다음 ID로 넘어감
    
            else:
                time.sleep(100)
        
        except Exception as e:
            # 다른 모든 경우에도 재시도
            time.sleep(10)

# 전체 데이터를 CSV로 저장
data_df.to_csv('data_collection.csv', index=False)


100%|██████████| 119569/119569 [175:13:09<00:00,  5.28s/it]        
